In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [2]:
%%fenicsx

import numpy as np
from mpi4py import MPI

import basix
import basix.ufl

from dolfinx import mesh, fem, plot
import ufl

import pyvista as pv

# Define mesh vertices (6 points in 2D)
x = np.array([[0.0, 0.0], [1.0, 0.0], [2.0, 0.0],
              [0.0, 1.0], [1.0, 1.0], [2.0, 1.0]], dtype=np.float32)

# Define cells (2 quadrilateral elements)
cells = np.array([[0, 1, 3, 4], [1, 2, 4, 5]], dtype=np.int64)

# Create coordinate element (Lagrange P1 on quadrilateral)
coordinate_element = basix.ufl.element("Lagrange", "quadrilateral", 1,
                                      shape=(x.shape[1],))

# Create mesh
msh = mesh.create_mesh(MPI.COMM_WORLD, cells, ufl.Mesh(coordinate_element), x)

# Create function space (Lagrange P1)
V = fem.functionspace(msh, ("Lagrange", 1))

# Save mesh as VTK format for ParaView
topology, cell_types, geometry = plot.vtk_mesh(msh, msh.topology.dim)
grid = pv.UnstructuredGrid(topology, cell_types, geometry)
grid.save("quadrilateral_mesh.vtk")

print("Mesh saved: quadrilateral_mesh.vtk")
print(f"Mesh dimension: {msh.topology.dim}")
print(f"Number of cells: {msh.topology.index_map(msh.topology.dim).size_local}")
print(f"Number of vertices: {msh.topology.index_map(0).size_local}")
print(f"Function space degree: {V.element.basix_element.degree}")
print(f"Function space dimension: {V.dofmap.index_map.size_local}")

Mesh saved: quadrilateral_mesh.vtk
Mesh dimension: 2
Number of cells: 2
Number of vertices: 6
Function space degree: 1
Function space dimension: 6


In [3]:
from google.colab import files
files.download('quadrilateral_mesh.vtk')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>